In [4]:
#import stuff
import os
import sys
import arrow
import numpy as np
import pandas as pd
from math import pi
import seaborn as sns
from datetime import datetime
from numpy import nan_to_num
import icartt

In [7]:
filename = 'C:/Users/cphal/Desktop/MAC prediction model/FIREXAQ-LARGE-CDP_DC8_20190722_R0.ict'

In [8]:
#function purpose: extract diameter input for FIREXAQ:
def extract_diameters_firexaq(filename, starting_string, bin_location, num_others_beginning,num_others_end, spacer):
    f = open(filename)
    nextline = f.readline()

    #extract and ignore the data we don't care about
    while '-9999' not in nextline:
        nextline = f.readline()

    #extract variables at the beginning that we don't care about
    for x in range(num_others_beginning):
        nextline = f.readline()

    #create list to store description of variables (to extract bin sizes from)
    list_bin_descriptions = []
    nextline = f.readline()
    while len(nextline) > 10:
        nextrow = nextline.split(',')
        list_bin_descriptions.append(nextrow)
        nextline = f.readline()
    
    #build list of bin sizes, exracting variables at the end that we don't care about
    bin_size_description = []
    for x in range(len(list_bin_descriptions)-num_others_end):
        bin_size_description.append(list_bin_descriptions[x][bin_location])

    #create new list with just the numbers
    bin_sizes = []
    
    for x in range(len(bin_size_description)):
        #extract beginning of description
        string_length = len(starting_string) + spacer
        #build number and add to list
        str1 = bin_size_description[x]
        str1.replace(" ", "_")
        c = str1[string_length]
        new_bin_size = ''
        while(c != 'u'):
            new_bin_size = new_bin_size + c
            string_length += 1
            c = str1[string_length]
        bin_sizes.append(float(new_bin_size))

    size_dist_diameter_input = pd.Series(bin_sizes)
    return size_dist_diameter_input        
    
    f.close()

#UNITS: um (find way to remember this and implement it)

In [ ]:
def extract_diameters_las(filename, starting_string, tag):
    f = open(filename)
    nextline = f.readline()

    #extract and ignore the data we don't care about
    while tag not in nextline:
        nextline = f.readline()

    #create new list with just the numbers
    list_row = nextline[len(starting_string):len(nextline)]
    bin_sizes = list_row.split(",")

    #delete unneeded ones
    bin_sizes.remove('\n')

    size_dist_diameter_input = pd.Series(bin_sizes)
    return size_dist_diameter_input 
    
    f.close()

filename = "C:/Users/cphal/Desktop/MAC prediction model/FIREXAQ-LARGE-LAScold_DC8_20190716_R0.ict"
tag = "OTHER_COMMENTS"
starting_string = "OTHER_COMMENTS: Midpoint diameters of Bins 00 to 79 determined by DMA-classified ammonium sulfate calibrations are: "
size_dist_diameter_input = extract_diameters_las(filename,starting_string,tag)     
print(size_dist_diameter_input) 

In [10]:
#purpose: import icartt data and convert to dataframe

#set directory
#fileDirectory = str('c:/Users/cphal/Desktop/NOAA_DATA/')
#prompt user to enter campaign name
#campaign = input('Campaign Name: ')
#campaign = 'firexaq'
#campaign = campaign.upper()

#get filenames in folder
#path = os.chdir(str(fileDirectory + campaign + '/'))
#folder_files = []
#with os.scandir(path) as entries:
    #for entry in entries:
        #folder_files.append(entry.name)

filename = 'C:/Users/cphal/Desktop/MAC prediction model/FIREXAQ-LARGE-CDP_DC8_20190722_R0.ict'
starting_string = 'number_size_distribution_dNdlogD_at_bin_center_'
bin_location = 3
num_others_beginning = 2
num_others_end = 0
spacer = 1

filename = 'C:/Users/cphal/Desktop/MAC prediction model/FIREXAQ-LARGE-CPSPD_DC8_20190722_R0.ict'
starting_string = ' bin center EQ '
bin_location = 4
num_others_beginning = 2
num_others_end = 1
spacer = 0

size_dist_diameter_input = extract_diameters_firexaq(filename, starting_string, bin_location, num_others_beginning, num_others_end, spacer)

#load in ict file to python
ict = icartt.Dataset(filename)
df = ict.data[:]
varnames = [x for x in ict.variables]
df = pd.DataFrame(df, columns = varnames)

c:\Users\cphal\AppData\Local\Programs\Python\Python310\lib\site-packages\icartt\dataset.py:290: UserWarning: Normal comments: required keyword INSTRUMENT_INFO is missing.
  warnings.warn(
c:\Users\cphal\AppData\Local\Programs\Python\Python310\lib\site-packages\icartt\dataset.py:692: UserWarning: Number of header lines suggested in line 1 (68) do not match actual header lines read (69)
  warnings.warn(


In [11]:
#put first column into datetime file; next two columns are nCDP and vCDP
#last columns are the bins - size_dist_input

#time after midnight from dataframe - may not be needed
#datetime = df['Time_mid']
#totalCDP = df[['nCDP','vCDP']]

binnames = varnames[num_others_beginning+1:len(varnames)-num_others_end]
size_dist_input_original = df[binnames]

In [12]:
#transpose size_dist_input to get it to work with Module A
size_dist_input = pd.DataFrame.transpose(size_dist_input_original)

In [13]:
#create new vector to hold column names
ser=pd.Series(range(np.size(size_dist_input,1)))
#assign this to be the columns
size_dist_input.rename(columns = ser)

,0,1,2,3,4,5,6,7,8,9,...,26047,26048,26049,26050,26051,26052,26053,26054,26055,26056
CPSPD_Bin01_stdPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CPSPD_Bin02_stdPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CPSPD_Bin03_stdPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CPSPD_Bin04_stdPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CPSPD_Bin05_stdPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CPSPD_Bin06_stdPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CPSPD_Bin07_stdPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CPSPD_Bin08_stdPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CPSPD_Bin09_stdPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CPSPD_Bin10_stdPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
#Implement Module A (w/o merge) (these are the dividing lines)
#Note: check units
d_N1 = int(10.5)
d_N2 = int(30)
d_V1 = int(25)
d_V2 = int(50)

In [3]:
#prompt use to enter the number of bins desired
num_bins = input("How many bins? ")

#create list based on that number to hold dividers
bin_divider_list = []
for x in range(int(num_bins)-1):
    next_divider = input("Please enter divider " + str(x+1) + ": ")
    bin_divider_list.append(next_divider)

In [14]:
date_time = ict.times
datetimedf = pd.DataFrame(date_time, columns= ['datetime'])
num1 = str(date_time[0])
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000000000')

In [ ]:
#calculate dVdlogdP using dNdlogdP
size_dist_volume = pd.DataFrame().reindex_like(size_dist_input)
for m in range(np.size(size_dist_volume,1)):
    for n in range(len(size_dist_diameter_input)):
        size_dist_volume.values[n,m]=pi/6* np.power(size_dist_diameter_input, 3).values[n]*size_dist_input.values[n,m]

In [18]:
#create dataframe to hold info
df1 = pd.DataFrame()
for x in range(len(num_bins)):
    df1['N' + str(x)] = np.zeros(np.size(size_dist_input,1))
print(df1)

        N0
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
...    ...
26052  0.0
26053  0.0
26054  0.0
26055  0.0
26056  0.0

[26057 rows x 1 columns]


In [57]:
N1 = np.zeros(np.size(size_dist_input,1)); N2 = np.zeros(np.size(size_dist_input,1)); N3 = np.zeros(np.size(size_dist_input,1))
F_N1 = np.zeros(np.size(size_dist_input,1)); F_N2 = np.zeros(np.size(size_dist_input,1)); F_N3 = np.zeros(np.size(size_dist_input,1))   
areaXY_number = np.zeros(np.size(size_dist_input,1))  
V1 = np.zeros(np.size(size_dist_input,1)); V2 = np.zeros(np.size(size_dist_input,1)); V3 = np.zeros(np.size(size_dist_input,1))
F_V1 = np.zeros(np.size(size_dist_input,1)); F_V2 = np.zeros(np.size(size_dist_input,1)); F_V3 = np.zeros(np.size(size_dist_input,1))   
areaXY_volume = np.zeros(np.size(size_dist_input,1))

In [58]:
#prepare for trapz calculation
size_dist_input[np.isnan(size_dist_input)] = 0 
size_dist_volume[np.isnan(size_dist_volume)] = 0 

ln_size_dist_diameter_input=np.log(size_dist_diameter_input)

In [59]:
#function purpose: find nearest value (d_N1, d_N2, d_V1, d_V2) in numpy array (instrument measured diameters)
def find_nearest(array, value):
    n = [abs(i-value) for i in array]
    idx = n.index(min(n))
    return idx

In [62]:
for m in range(np.size(size_dist_input,1)):

        d_N1_nearest = find_nearest(size_dist_diameter_input,d_N1)
        d_N2_nearest = find_nearest(size_dist_diameter_input,d_N2)
        d_V1_nearest = find_nearest(size_dist_diameter_input,d_V1)
        d_V2_nearest = find_nearest(size_dist_diameter_input,d_V2)
        areaXY_number[m] = np.trapz(size_dist_input.values[:,m], x=ln_size_dist_diameter_input)
        N1[m] = np.trapz(size_dist_input.values[0:d_N1_nearest,m], x=ln_size_dist_diameter_input[0:d_N1_nearest])
        N2[m] = np.trapz(size_dist_input.values[d_N1_nearest+1:d_N2_nearest,m], x=ln_size_dist_diameter_input[d_N1_nearest+1:d_N2_nearest])    
        N3[m] = np.trapz(size_dist_input.values[d_N2_nearest+1:,m], x=ln_size_dist_diameter_input[d_N2_nearest+1:])
        F_N1[m] = N1[m]/areaXY_number[m]
        F_N2[m] = N2[m]/areaXY_number[m]       
        F_N3[m] = N3[m]/areaXY_number[m]

        areaXY_volume[m] = np.trapz(size_dist_volume.values[:,m], x=ln_size_dist_diameter_input)
        V1[m] = np.trapz(size_dist_volume.values[0:d_V1_nearest,m], x=ln_size_dist_diameter_input[0:d_V1_nearest])
        V2[m] = np.trapz(size_dist_volume.values[d_V1_nearest+1:d_V2_nearest,m], x=ln_size_dist_diameter_input[d_V1_nearest+1:d_V2_nearest])    
        V3[m] = np.trapz(size_dist_volume.values[d_V2_nearest+1:,m], x=ln_size_dist_diameter_input[d_V2_nearest+1:])
        F_V1[m] = V1[m]/areaXY_volume[m]
        F_V2[m] = V2[m]/areaXY_volume[m]       
        F_V3[m] = V3[m]/areaXY_volume[m]

C:\Users\cphal\AppData\Local\Temp\ipykernel_5460\1990599437.py:11: RuntimeWarning: invalid value encountered in double_scalars
  F_N1[m] = N1[m]/areaXY_number[m]
C:\Users\cphal\AppData\Local\Temp\ipykernel_5460\1990599437.py:12: RuntimeWarning: invalid value encountered in double_scalars
  F_N2[m] = N2[m]/areaXY_number[m]
C:\Users\cphal\AppData\Local\Temp\ipykernel_5460\1990599437.py:13: RuntimeWarning: invalid value encountered in double_scalars
  F_N3[m] = N3[m]/areaXY_number[m]
C:\Users\cphal\AppData\Local\Temp\ipykernel_5460\1990599437.py:19: RuntimeWarning: invalid value encountered in double_scalars
  F_V1[m] = V1[m]/areaXY_volume[m]
C:\Users\cphal\AppData\Local\Temp\ipykernel_5460\1990599437.py:20: RuntimeWarning: invalid value encountered in double_scalars
  F_V2[m] = V2[m]/areaXY_volume[m]
C:\Users\cphal\AppData\Local\Temp\ipykernel_5460\1990599437.py:21: RuntimeWarning: invalid value encountered in double_scalars
  F_V3[m] = V3[m]/areaXY_volume[m]


In [63]:
# If the selected size classes are broader than the measured aerosol sizes 
print (d_N1,d_N2,d_V1,d_V2,size_dist_diameter_input[len(size_dist_diameter_input)-1])
if (d_N1 < size_dist_diameter_input[0]):  #only large particles is available
    N1=0; F_N1=0  
    print("error: size classes")
if (d_N2 > size_dist_diameter_input[len(size_dist_diameter_input)-1]):  #only small particles is available
    N3=0; F_N3=0
    print("error: size classes")
if (d_V1 < size_dist_diameter_input[0]):
    V1=0; F_V1=0
    print("error: size classes")
if (d_V2 > size_dist_diameter_input[len(size_dist_diameter_input)-1]):
    V3=0; F_V=0
    print("error: size classes")

10 30 25 50 49.0


In [64]:
file_M_1 = pd.DataFrame() 
file_M_1['datetime'] = datetimedf
file_M_1['N1'] = N1; file_M_1['N2'] = N2 ; file_M_1['N3'] = N3 ; file_M_1['N_total'] = areaXY_number 
file_M_1['F_N1'] = F_N1; file_M_1['F_N2'] = F_N2 ; file_M_1['F_N3'] = F_N3 
file_M_1['V1'] = V1; file_M_1['V2'] = V2 ; file_M_1['V3'] = V3 ; file_M_1['V_total'] = areaXY_volume 
file_M_1['F_V1'] = F_V1; file_M_1['F_V2'] = F_V2 ; file_M_1['F_V3'] = F_V3 

In [65]:
print(file_M_1)

                 datetime        N1   N2   N3   N_total  F_N1  F_N2  F_N3  \
0     2019-07-22 18:15:40  0.000000  0.0  0.0  0.000000   NaN   NaN   NaN   
1     2019-07-22 18:15:41  0.000000  0.0  0.0  0.000000   NaN   NaN   NaN   
2     2019-07-22 18:15:42  0.000000  0.0  0.0  0.000000   NaN   NaN   NaN   
3     2019-07-22 18:15:43  0.000000  0.0  0.0  0.000000   NaN   NaN   NaN   
4     2019-07-22 18:15:44  0.000000  0.0  0.0  0.000000   NaN   NaN   NaN   
...                   ...       ...  ...  ...       ...   ...   ...   ...   
24741 2019-07-23 01:08:01  0.007244  0.0  0.0  0.007244   1.0   0.0   0.0   
24742 2019-07-23 01:08:02  0.000000  0.0  0.0  0.000000   NaN   NaN   NaN   
24743 2019-07-23 01:08:03  0.204961  0.0  0.0  0.204961   1.0   0.0   0.0   
24744 2019-07-23 01:08:04  0.007671  0.0  0.0  0.007671   1.0   0.0   0.0   
24745 2019-07-23 01:08:05  0.221747  0.0  0.0  0.221747   1.0   0.0   0.0   

              V1   V2  V3    V_total  F_V1  F_V2  F_V3  
0       0.000000  